## Set up

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows',200)

## Load queried tweets

In [2]:
# load queried tweets
file_in = '../data/derived/tweets_query.csv'
tweet_df = pd.read_csv(file_in)

# preview tweets_query.csv
tweet_df.head()

,tweet_id,text,label
0,597576902212063232,Cisco had to deal with a fat cash payout to th...,0.0
1,565586175864610817,"@MadamPlumpette I'm decent at editing, no worr...",0.0
2,563881580209246209,@girlziplocked will read. gotta go afk for a b...,0.0
3,595380689534656512,guys. show me the data. show me your github. t...,0.0
4,563757610327748608,@tpw_rules nothings broken. I was just driving...,0.0


## Check sample size of each feature

In [3]:
len(tweet_df)

17504

There are 17504 records from the original datasets combined.

In [4]:
tweet_df.count()

tweet_id    17504
text        11310
label       17481
dtype: int64

In [5]:
tweet_df.count() / len(tweet_df)

tweet_id    1.000000
text        0.646138
label       0.998686
dtype: float64

All records have tweet IDs, over half had successfully queried text, and almost all had a label.
* tweet_id: all 17504 tweets (100%) from the datasets have a tweet_id.
* text: 11310 tweets (64%) from the datasets had a successfully queried tweet.
* label: 17481 tweets (99%) have a label.

## Check sample size of successfully queried tweets by label

In [6]:
# count sample size by label from queried tweets
count_at_load = tweet_df['label'].value_counts(dropna=False).sort_index()
# initialize dataframe to track sample size by label throughout cleaning
track_size_df = pd.DataFrame(data={'count_at_load':count_at_load})
# preview dataframe
track_size_df

,count_at_load
0.0,5850
1.0,98
2.0,911
3.0,50
4.0,3377
5.0,7195
NaN,23


## Drop tweets without successfully queried text and without label

In [7]:
# drop tweets with NA label or text
tweet_df.dropna(inplace=True)

This step drops 6171 records without successfully queried tweets and drops 23 records without labels.

## Check sample size of successfully queried tweets with label

In [8]:
# count sample size by label after drop NA
count_at_drop_na = tweet_df['label'].value_counts(dropna=False).sort_index()
track_size_df['count_at_drop_na'] = count_at_drop_na
track_size_df

,count_at_load,count_at_drop_na
0.0,5850,5411.0
1.0,98,64.0
2.0,911,561.0
3.0,50,24.0
4.0,3377,2716.0
5.0,7195,2512.0
NaN,23,NaN


## Check for redundant tweets

In [9]:
sum(tweet_df.duplicated())

0

There are no duplicated rows based on all fields (tweet ID, text and label). This means there are no tweets with the same ID, label and text.

In [10]:
sum(tweet_df.duplicated(subset='tweet_id'))

95

There are 95 duplicated rows based on tweet ID. This means there are tweets with the same tweet ID.

In [11]:
sum(tweet_df.duplicated(subset=['tweet_id','text']))

95

There are 95 duplicated rows based on tweet ID and text. Since there are 0 duplicated rows based on all three features, these 95 tweets must have the same text and ID but different labels. This suggests they are tweets included in both the Waseem 2016 and Jha Mamidid 2017 dataset.

In [12]:
sum(tweet_df.duplicated(subset=['text','label']))

1907

There are 1907 duplicated rows based on text and label. This means there are tweets that had different tweet IDs, but the same text and label. These will be dropped because they do not add new information to our training data.

In [13]:
sum(tweet_df.duplicated(subset=['text']))

2002

There are 2002 duplicate rows based on text. This is a combination of the 95 tweets that were redundant between datasets, and the 1907 tweets that were redundant within the original datasets.

## Drop redundant tweets

#### Drop redundant tweets with the same text and label

In [14]:
# drop records with identical text and labels
tweet_df.drop_duplicates(subset=['text','label'], inplace=True)

Now, let's take a closer look at tweets with the same text but different IDs and labels.

In [15]:
# create column that indicates weather tweet text is duplicated
tweet_df['is_duplicated'] = tweet_df.duplicated(subset=['text'],keep=False)
# preview duplicated records
tweet_df[tweet_df['is_duplicated']][['tweet_id','text','label']].sort_values('text')

,tweet_id,text,label
6975,572330495830188032,"""@NewIdeamagazine:They weren't kidding when th...",4.0
3049,572330495830188032,"""@NewIdeamagazine:They weren't kidding when th...",2.0
5483,572337676482838528,"""Classy and elegant"" is one way of putting it....",0.0
7008,572337676482838528,"""Classy and elegant"" is one way of putting it....",4.0
6954,572318519464865792,"""It's Katie and Nikki to a T. In that it's tra...",4.0
165,572318519464865792,"""It's Katie and Nikki to a T. In that it's tra...",2.0
2029,572337021588414464,"""We want everyone to know we are not just pret...",2.0
7016,572337021588414464,"""We want everyone to know we are not just pret...",4.0
1711,572337067436351489,#MKR anyone can cook from a can girls.,2.0
7015,572337067436351489,#MKR anyone can cook from a can girls.,4.0


These are all tweets that were redundant between Waseem 2016 (which has labels 0 through 3) and Jha Mamidi (which has labels 4 through 5). There isn't perfect agreement as to whether tweets were sexist between the two datsets - all labels from Jha Mamidi 2017 are (4) hostile, while labels from Waseem 2016 are either (0) not sexist or (2) sexist. I'm going to pick the more detailed labels of sexism type from the Jha Mamidi dataset, and drop the repeats from Waseem 2016. Since I loaded the Jha Mamidi tweets after the Waseem tweets when I queried the data, I can simply keep the latter of redundant tweets to ensure I keep labels from Jha Mamidi.

In [16]:
# drop duplicated records with same text, keeping latter records
tweet_df.drop_duplicates(subset=['tweet_id','text'], keep='last', inplace=True)

In [17]:
# check for any remaining duplicates based on text
sum(tweet_df.duplicated(subset=['text']))

0

Perfect, no more duplicates remain.

## Check sample size after dropping duplicates

In [18]:
# count sample size by label after drop repeats
count_at_drop_repeat = tweet_df['label'].value_counts(dropna=False).sort_index()
track_size_df['count_at_drop_repeat'] = count_at_drop_repeat
# preview sample size
track_size_df

,count_at_load,count_at_drop_na,count_at_drop_repeat
0.0,5850,5411.0,5383.0
1.0,98,64.0,64.0
2.0,911,561.0,492.0
3.0,50,24.0,24.0
4.0,3377,2716.0,2716.0
5.0,7195,2512.0,607.0
NaN,23,NaN,NaN


## Add total and percent to counts

In [19]:
# sum total sample size at each step
track_size_df.loc['total',:] = np.sum(track_size_df)
# preview sample size
track_size_df

,count_at_load,count_at_drop_na,count_at_drop_repeat
0.0,5850.0,5411.0,5383.0
1.0,98.0,64.0,64.0
2.0,911.0,561.0,492.0
3.0,50.0,24.0,24.0
4.0,3377.0,2716.0,2716.0
5.0,7195.0,2512.0,607.0
NaN,23.0,NaN,NaN
total,17504.0,11288.0,9286.0


In [20]:
# calculate percent of sample size by label at each step of the data cleaning

# initialize empty list to track column order, alternating count and percent throughout steps
new_col_order = []

# iterate through columns
for col in track_size_df.columns:
    
    # store step name
    suffix = col[5:]
    # add percent before step name
    percent_col = 'percent' + suffix
    # add column names to list
    new_col_order.append(col)
    new_col_order.append(percent_col)
    
    # calculate percent of dataset at each label
    track_size_df[percent_col] = (track_size_df[col] / track_size_df.loc['total',col]) * 100

# reorder columns
track_size_df = track_size_df[new_col_order]

# preview sample size and percentages at each step
track_size_df

,count_at_load,percent_at_load,count_at_drop_na,percent_at_drop_na,count_at_drop_repeat,percent_at_drop_repeat
0.0,5850.0,33.420932,5411.0,47.935861,5383.0,57.968986
1.0,98.0,0.559872,64.0,0.566974,64.0,0.689210
2.0,911.0,5.204525,561.0,4.969880,492.0,5.298299
3.0,50.0,0.285649,24.0,0.212615,24.0,0.258454
4.0,3377.0,19.292733,2716.0,24.060950,2716.0,29.248331
5.0,7195.0,41.104890,2512.0,22.253721,607.0,6.536722
NaN,23.0,0.131399,NaN,NaN,NaN,NaN
total,17504.0,100.000000,11288.0,100.000000,9286.0,100.000000


## Write clean dataset and sample size tracker to CSV

In [21]:
# write clean tweet dataset to CSV
tweet_filepath_out = '../data/derived/tweets_clean.csv'
tweet_df.to_csv(path_or_buf=tweet_filepath_out, index=False)

# write sample size tracker to CSV
size_filepath_out = '../data/derived/track_size_clean.csv'
track_size_df.to_csv(path_or_buf=size_filepath_out, index=False)